In [ ]:
# Importamos librerías externas: 

# Para arrancar el proyecto, necesitamos nuestras herramientas principales, que son las librerías que amplían las capacidades de Python.
# Las librerías son herramientas creadas por la comunidad que amplían Python y se instalan con pip.

# Conectar y manejar Bases de Datos MySQL. (Necesario para Fases 2 y 3)
import mysql.connector

# El motor matemático: maneja números y cálculos a gran velocidad. (Útil, aunque no esencial para este ejercicio porque no haremos cálculos complejos)
import numpy as np

# El Excel de Python: organiza y analiza datos en forma de tablas (DataFrames). (Esencial para guardar las películas)
import pandas as pd

# para poder visualizar todas las columnas de los DataFrames (Muy útil cuando trabajas con DataFrames de Pandas que tienen muchas columnas)
pd.set_option('display.max_columns', None) 

# El cartero de Internet: pide y envía información a páginas web y servidores. (Esencial para descargar la API)
import requests

# Manual de "Qué hacer si falla": identifica problemas al conectar a MySQL. (Buena práctica para manejar errores)
from mysql.connector import Error